In [1]:
import math
from collections import namedtuple

import colander
from colander import Invalid

math.max = max
math.min = min

In [2]:
Dimensions = namedtuple("Dimensions", "length, width, height")

In [3]:
class DimensionsType(colander.SchemaType):
    def serialize(self, node, appstruct):
        if appstruct is colander.null:
            return colander.null
        if not isinstance(appstruct, Dimensions):
            raise Invalid(node, f"{appstruct!r} is not a Dimensions type")
        return f"{appstruct.length}x{appstruct.width}x{appstruct.height}"
    
    def deserialize(self, node, cstruct):
        if cstruct is colander.null:
            return colander.null
        if isinstance(cstruct, Dimensions):
            return cstruct
        try:
            result = str(cstruct)
        except Exception:
            raise Invalid(node, f"{cstruct} is not a string")
        try:
            l, w, h = result.split('x')
        except ValueError:
            raise Invalid(node, f"{cstruct} does not fit the pattern <length>x<width>x<height>")
        try:
            return Dimensions(float(l), float(w), float(h))
        except ValueError:
            raise Invalid(node, f"{cstruct} contains invalid numbers")

In [4]:
def LargestDimensionRange(min=None, max=None):
    def validator(node, value):
        largest = math.max(value)
        if min is not None and largest < min:
            raise Invalid(node, f"largest dimension of {value!r} is less than minimum value {min}")
        if max is not None and largest > max:
            raise Invalid(node, f"largest dimensions of {value!r} is greater than maximum value {max}")
    return validator

In [5]:
def VolumeRange(min=None, max=None):
    def validator(node, value):
        volume = value.length * value.width * value.height
        if min is not None and volume < min:
            raise Invalid(node, f"volume of {value!r} is less than minimum value {min}")
        if max is not None and volume > max:
            raise Invalid(node, f"volume of {value!r} is greater than maximum value {max}")
    return validator

In [6]:
schema = colander.SchemaNode(
    DimensionsType(),
    validator=colander.All(
        LargestDimensionRange(max=10),
        VolumeRange(max=250)
    ),
    name='dimensions'
)

In [7]:
schema.deserialize(Dimensions(8, 5, 6))

Dimensions(length=8, width=5, height=6)

In [8]:
schema.deserialize("8x5x6")

Dimensions(length=8.0, width=5.0, height=6.0)

In [9]:
try:
    schema.deserialize(Dimensions(11, 0.5, 0.3))
except colander.Invalid as e:
    print(e)

{'dimensions': 'largest dimensions of Dimensions(length=11, width=0.5, '
               'height=0.3) is greater than maximum value 10'}


In [10]:
try:
    schema.deserialize(Dimensions(8, 5, 7))
except colander.Invalid as e:
    print(e)

{'dimensions': 'volume of Dimensions(length=8, width=5, height=7) is greater '
               'than maximum value 250'}
